In [1]:
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats

In [2]:
df = pd.read_json('../data/wikidata-movies.json.gz', orient='records', lines=True)
df2 = pd.read_json('../data/wikidata-movies2.json.gz', orient='records', lines=True)
genres = pd.read_json('../data/genres.json.gz', orient='records', lines=True)

FileNotFoundError: [Errno 2] No such file or directory: '../data/wikidata-movies.json.gz'

In [ ]:
df = df[pd.notnull(df['made_profit'])]

In [ ]:
df['genre'] = df['genre'].str[0]
genres.columns = ['genre_label', 'genre']

In [ ]:
merged = pd.merge(df, genres, on="genre")

In [ ]:
df_stack = df['genre'].apply(pd.Series).stack().reset_index(level=1, drop=True).to_frame()
df_stack.columns = ['wikidata_id']

In [ ]:
df_genre = df_stack.reset_index().merge(genres, how="left", on="wikidata_id").set_index('index')

In [ ]:
df_genre = pd.merge(df_genre, df, left_index=True, right_index=True).reset_index()

In [ ]:
genres['profit'] = genres['genre_label'].apply(lambda x: df_genre[(df_genre['genre_label'] == x) & (df_genre['made_profit'] == True)].shape[0])
genres['noprofit'] = genres['genre_label'].apply(lambda x: df_genre[(df_genre['genre_label'] == x) & (df_genre['made_profit'] == False)].shape[0])

In [ ]:
df_genre[(df_genre['country_of_origin'] == 'Q29') & (df_genre['made_profit'] == True)].shape[0]

In [ ]:
df_genre = genres.sort_values(by='profit', ascending=False)
df_genre = df_genre[(df_genre['profit'] != 0) & (df_genre['noprofit'] != 0)]

In [ ]:
# profit_percent = profit/total
df_genre['profit_percent'] = df_genre['profit'] / (df_genre['profit'] + df_genre['noprofit'])

In [ ]:
df_genre['profit']

In [ ]:
contingency = np.vstack([df_genre['profit'], df_genre['noprofit']])

In [ ]:
chi2, p, dof, expected = stats.chi2_contingency(contingency)

In [ ]:
p

In [ ]:
df2 = df2[pd.notnull(df2['nbox'])]